In [1]:
# from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
# from langchain.llms import CTransformers
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import time

In [2]:
# load PINECONE_API_KEY from .env
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Load data

In [3]:
# load data from '/Data/'
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    files = loader.load()
    return files

In [4]:
documents = load_pdf("../Data/")

Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEnc

In [5]:
# documents

In [6]:
len(documents)

871

# Prepare Data

In [7]:
documents[0]

Document(metadata={'source': '../Data/standard-treatment-guidelines.pdf', 'page': 0, 'page_label': '1'}, page_content='i\nSTANDARD TREATMENT GUIDELINES\nA Manual for Medical Therapeutics\nFirst Edition, 2013\nGujarat Medical Services Corporation Limited\nHealth & Family Welfare Department\nGovernment of Gujarat')

In [8]:
documents[0].page_content

'i\nSTANDARD TREATMENT GUIDELINES\nA Manual for Medical Therapeutics\nFirst Edition, 2013\nGujarat Medical Services Corporation Limited\nHealth & Family Welfare Department\nGovernment of Gujarat'

In [9]:
documents[1].page_content

'ii\nMessage\n(Health Minister)'

In [10]:
for i in range(10):
    print(documents[i].page_content)

i
STANDARD TREATMENT GUIDELINES
A Manual for Medical Therapeutics
First Edition, 2013
Gujarat Medical Services Corporation Limited
Health & Family Welfare Department
Government of Gujarat
ii
Message
(Health Minister)
iii
Message
(Principal Secretary)
iv
Message
(Commissionerate)
v
PREFACE TO FIRST EDITION
The Rational Use of Medicines results into a more effective treatment at an affordable cost.
Essential Medicines List (EML) and the Standard Treatment Guideline s (STG) are an
important component of promoting rational use of medicines. Gujarat Medical Service
Corporation Limited under the auspices of Department of Health & Family Welfare,
Government of Gujarat finalizes EML and has undertaken the task to develop S TG for the
State. The objective is to improve the availability of quality drugs at each health facility and
to achieve maximum public health improvement using available resources. B. J. Medical
College & Civil Hospital, Ahmedabad has been instrumental in t he preparation of 

In [11]:
combined_text = " ".join(doc.page_content for doc in documents if doc.page_content)

import re
def normalize_text(text):
    re_text = re.sub(r"\s+", " ", text)
    re_text.strip()
    return re_text

print(combined_text[:200])
combined_text = normalize_text(combined_text)
print(combined_text[:200])


i
STANDARD TREATMENT GUIDELINES
A Manual for Medical Therapeutics
First Edition, 2013
Gujarat Medical Services Corporation Limited
Health & Family Welfare Department
Government of Gujarat ii
Message
(
i STANDARD TREATMENT GUIDELINES A Manual for Medical Therapeutics First Edition, 2013 Gujarat Medical Services Corporation Limited Health & Family Welfare Department Government of Gujarat ii Message (


# Create Chunks

In [12]:
# create chunks
# while creating chunk we have 871 list of documents which will loose the context while splitting the text
# to solve this issue we need to first combine the documents into one document then pass it to text_splitter

# create document

def create_chunks(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
    chunks = text_splitter.split_text(data)
    return chunks

chunks = create_chunks(combined_text)

In [13]:
a = chunks[0][-20:]
print(chunks[0])
print(a)

i STANDARD TREATMENT GUIDELINES A Manual for Medical Therapeutics First Edition, 2013 Gujarat
dition, 2013 Gujarat


In [14]:
b = chunks[1][:20]
print(chunks[1])
print(b)

2013 Gujarat Medical Services Corporation Limited Health & Family Welfare Department Government of
2013 Gujarat Medical


In [15]:
a == b

False

    The above statement is showing a == b as False, but if we examine the values of a and b, we notice that certain words match between them. When creating chunks, the text_splitter does not overlap chunks character by character. Instead, it attempts to find logical splitting points (such as at spaces or word boundaries) while maintaining the specified chunk size and overlap. This means the overlap is based on complete words rather than individual characters, ensuring that chunks remain meaningful and contextually relevant.

In [16]:
print(len(chunks))

21350


In [17]:
combined_text[:100]

'i STANDARD TREATMENT GUIDELINES A Manual for Medical Therapeutics First Edition, 2013 Gujarat Medica'

In [18]:
# Debug: Check if overlaps exist
for i in range(20):
    overlap = chunks[i][-20:]
    next_chunk_start = chunks[i + 1][:20] 
    print(f"Chunk {i} ends with: {overlap}")
    print(f"Chunk {i + 1} starts with: {next_chunk_start}")
    print(f"Overlap valid: {overlap == next_chunk_start}")
    print("--------------------------------------------------------------------------------------")

Chunk 0 ends with: dition, 2013 Gujarat
Chunk 1 starts with: 2013 Gujarat Medical
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 1 ends with: rtment Government of
Chunk 2 starts with: Government of Gujara
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 2 ends with: ecretary) iv Message
Chunk 3 starts with: iv Message (Commissi
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 3 ends with: dicines results into
Chunk 4 starts with: results into a more 
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 4 ends with: cines List (EML) and
Chunk 5 starts with: List (EML) and the S
Overlap valid: False
--------------------------------------------------------------------------------------
Chunk 5 ends with: mponent of promoting
Chunk 6 st

In [19]:
# we normalized the combined_text above because many of our chunks does not getting overlapping sequences.

# Embedding

In [20]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [21]:
embedding_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [22]:
# we can use both SentenceTransformer and Pinecode to embed the chunks. First one use SentenceTransformer
# and below commented part is using pinecone

embeddings = embedding_model.encode(chunks, batch_size=32, show_progress_bar=True)

# use pinecone embedding technique when we need to store the embeddings on hosted service of pinecone
# embeddings = pc.inference.embed(
#     model='multilingual-e5-large',
#     inputs=chunks,
#     parameters={"input_type": "passage", "truncate": "END"}
# )

Batches:   0%|          | 0/668 [00:00<?, ?it/s]

In [23]:
EMBEDDING_MODEL_DIMENSION = embedding_model[1].word_embedding_dimension
print(EMBEDDING_MODEL_DIMENSION)

384


In [24]:
# embeddings = embedding_model.encode(chunks[0])
# print(embeddings)

In [25]:
pc = Pinecone(api_key=PINECONE_API_KEY) # initialize pinecone

In [26]:
index_name = 'medical-chatbot'

In [27]:
if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=EMBEDDING_MODEL_DIMENSION,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

while not pc.describe_index(index_name).status["ready"] == True:
    time.sleep(1)

In [28]:
pc.describe_index(index_name)

{
    "name": "medical-chatbot",
    "dimension": 384,
    "metric": "cosine",
    "host": "medical-chatbot-j5amzap.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [29]:
index = pc.Index(index_name)

In [30]:
 # list all record listed in pinecone database
# ids_list = index.list(namespace="medical-knowledge") # This will contain the list of ids(eg. chunk-0, chunk-1,...)

In [31]:
# store ids in the form of list to get faster search
# ids_dict

In [32]:
embeddings[0]

array([-3.04512661e-02,  8.53797048e-03, -4.35634851e-02, -1.45671759e-02,
       -1.33523107e-01,  7.73802772e-03, -4.05765139e-02,  1.35740533e-01,
       -4.01867330e-02, -1.78049784e-02,  1.62634011e-02,  5.33752069e-02,
       -2.01637857e-02,  5.21901362e-02,  1.43679203e-02, -3.64299268e-02,
        6.83173612e-02, -4.64110859e-02,  5.43257371e-02,  2.45430265e-02,
        5.87754697e-02,  9.10631269e-02,  4.21114266e-02, -2.84499805e-02,
       -5.41715175e-02, -1.63956974e-02, -5.39367832e-02, -1.98082756e-02,
        4.94260266e-02, -1.81078818e-02, -2.98382109e-03,  3.32461707e-02,
        7.14892298e-02, -6.62621707e-02, -7.94173963e-03,  3.89257967e-02,
       -2.57469583e-02,  1.53554708e-03, -4.59754728e-02,  4.65780124e-02,
        4.28945757e-02, -2.30956189e-02, -3.34977694e-02, -6.73370715e-03,
        5.96111231e-02, -2.79240049e-02, -1.21782035e-01,  4.56850231e-03,
        7.30696553e-03,  6.32041842e-02, -8.62979963e-02, -9.36403312e-03,
        1.80958398e-02,  

In [33]:
index.query(
    namespace="medical-knowledge",
    vector=embeddings[0].tolist(),
    top_k=3,
    include_metadata=True
)

{'matches': [{'id': 'chunk-1',
              'metadata': {'text': 'i STANDARD TREATMENT GUIDELINES A Manual '
                                   'for Medical Therapeutics First Edition, '
                                   '2013 Gujarat'},
              'score': 1.0000248,
              'values': []},
             {'id': 'chunk-41',
              'metadata': {'text': 'The standard treatment guidelines have '
                                   'been developed a fter several meetings and '
                                   'discussions among'},
              'score': 0.677619576,
              'values': []},
             {'id': 'chunk-14972',
              'metadata': {'text': 'before starting treatment. Page 193 / 409 '
                                   '1. Pan American Health Organization. '
                                   'Guidelines for'},
              'score': 0.655840099,
              'values': []}],
 'namespace': 'medical-knowledge',
 'usage': {'read_units': 6}}

In [34]:
# # prepare record for upsert
# records = []

# for i, (chunk, embed) in enumerate(zip(chunks, embeddings)):
#     q = index.query(
#         namespace="medical-knowledge",
#         vector=embed.tolist(),
#         top_k=3,
#         include_metadata=True
#     )
    
#     if not any(match["metadata"]["text"] == chunk for match in q.matches):
#         records.append(
#             {
#                 "id": f"chunk-{i+1}",
#                 "values": embed.tolist(),
#                 "metadata": {"text": chunk}
#             }
#         )
#         print(chunk)

# # Upsert records in batches if needed to avoid payload size issue
# index.upsert(
#     vectors=records,
#     namespace='medical-knowledge'
# )

In [35]:
# Step 1: Retrieve all existing IDs in the namespace
# existing_ids = set()  # Use a set for O(1) lookup
# response = index.describe_index_stats(namespace='medical-knowledge')

# print(response)

In [36]:
# if 'namespaces' in response and 'medical-knowledge' in response['namespaces']:
#     existing_ids = set(response['namespaces']['medical-knowledge']['Metadata']['ids'])

# # Step 2: Filter records to exclude those with existing IDs
# filtered_records = [record for record in records if record['id'] not in existing_ids]

# # Step 3: Batch insert only new records
# batch_size = 100
# for i in range(0, len(filtered_records), batch_size):
#     index.upsert(
#         vectors=filtered_records[i:i + batch_size],
#         namespace='medical-knowledge'
#     )

In [37]:
# len(records)

## SEARCH THE INDEX

In [38]:
user_query = "What is dengue?"

# convert the query into the numerical vector so that pinecone can search with
query_embedding = embedding_model.encode(user_query, convert_to_numpy=True)

In [39]:
print(query_embedding.tolist())

[-0.01418101042509079, 0.007480910047888756, -0.07430920004844666, 0.013342592865228653, 0.021969908848404884, 0.03531714156270027, 0.10833455622196198, 0.033305056393146515, 0.006316029466688633, -0.027727434411644936, 0.049590352922677994, -0.021513180807232857, -0.03149483725428581, -0.026573700830340385, -0.015978418290615082, 0.008288472890853882, -0.027085430920124054, -0.08138439804315567, 0.0014592327643185854, -0.0564335398375988, -0.027822345495224, 0.08596830815076828, 0.015447396785020828, -0.08145946264266968, -0.09959868341684341, 0.04570760577917099, -0.03021002747118473, -0.003603380871936679, 0.05652910843491554, -0.06074589118361473, 0.022609328851103783, 0.06152968108654022, 0.016123250126838684, 0.029100924730300903, 0.0026446415577083826, 0.0012485277839004993, -0.050113119184970856, 0.038598254323005676, -0.000509895384311676, 0.031377363950014114, 0.035161715000867844, 0.005384853109717369, 0.023384181782603264, -0.05903847888112068, 0.006182401441037655, 0.04171

In [40]:
# Rank result
# search the index for the k most similar vectors
result = index.query(
    namespace="medical-knowledge",
    vector=query_embedding.tolist(),
    top_k=3,
    include_metadata=True,
    include_values=False
)

print(result)

{'matches': [{'id': 'chunk-16642',
              'metadata': {'text': 'infection may be asymptomatic or present '
                                   'as mild or occasionally severe dengue '
                                   'fever. Subsequent'},
              'score': 0.79547894,
              'values': []},
             {'id': 'chunk-16643',
              'metadata': {'text': 'fever. Subsequent infections increase the '
                                   'risk of severe dengue. Clinical features '
                                   'After the'},
              'score': 0.785868466,
              'values': []},
             {'id': 'chunk-789',
              'metadata': {'text': 'Dengue is the most important emerging '
                                   'tropical viral disease of human beings in '
                                   'the world today.'},
              'score': 0.778169274,
              'values': []}],
 'namespace': 'medical-knowledge',
 'usage': {'read_units': 6}}


    Now we will give this data(result) to our large language model EleutherAI/gpt-neo-1.3B, it will read the question and answer given by the embedding model and answer it.

    RESULT: If we see the above result for the given result we will find that our model is working really good as above two chunks of id 'chunk-16642' and 'chunk-16643', both giving the symptoms and third chunk with id 'chunk-789' tells about the dengue. So our result is working fine.

In [41]:
# PROMPT = PromptTemplate(template=prompt_template, input_variables=["User", "Response"])
# chain_type_kwargs={"prompt": PROMPT}

In [42]:
# Initialize the LLM using ctransformers
# will not work because we need model in tensorflow as CTransformer work with tensor data
# we need to convert the model parameter from pytorch format to tensor (GGML) format.

# llm = CTransformers(
#     model="/home/user/.cache/huggingface/hub/models--EleutherAI--gpt-neo-1.3B",
#     model_type="gpt_neo",
#     config={"max_new_tokens": 200, "temperature": 0.7, "top_p": 0.9}
# )

### Function to extract data from pinecone database

In [43]:
class UserQuery:
    def __call__(self):
        user_input = input("Enter your questions: ")
        return user_input
    def __str__(self):
        return "Type your query and hit enter"

user_query_object = UserQuery()
print(user_query_object)
query = user_query_object()
print(f"Query: {query}")

Type your query and hit enter


Enter your questions:  what is polio


Query: what is polio


In [44]:
def fetch_relevant_info(user_query):
    user_embedding = embedding_model.encode(user_query, convert_to_numpy=True)

    result = index.query(
        namespace="medical-knowledge",
        vector=user_embedding.tolist(),
        top_k=3,
        include_metadata=True,
        include_values=False
    )

    return result

In [45]:
search_results = fetch_relevant_info(query)
print(search_results)

{'matches': [{'id': 'chunk-16334',
              'metadata': {'text': 'is an acute viral infection due to a '
                                   'poliovirus (serotypes 1, 2 or 3). '
                                   'Human-to-human transmission'},
              'score': 0.601843655,
              'values': []},
             {'id': 'chunk-16394',
              'metadata': {'text': '08 June 2021] 3. Global Polio Eradication '
                                   'Initiative. Standard operating procedures: '
                                   'responding to'},
              'score': 0.598207414,
              'values': []},
             {'id': 'chunk-10184',
              'metadata': {'text': 'of Family Welfare . Salient features '
                                   'Treatment for acute polio cases All cases '
                                   'should be treated as'},
              'score': 0.581666708,
              'values': []}],
 'namespace': 'medical-knowledge',
 'usage': {'read_unit

# MODELLING

In [46]:
# model_name = "emilyalsentzer/Bio_ClinicalBERT"
model_name = "EleutherAI/gpt-neo-1.3B"

In [47]:
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [48]:
# prompt_template = """You are a responsible, safety-first AI assistant designed to provide general information and guidance on user_query and Relevant Information given below.
# Your role is to assist users with general medical knowledge. Follow the below format to answer the user.
# Formate:
# User Query: {user_query}
# Relevant Information: {query_results}
# AI Response:
# """

In [59]:
query = "what is polio?"

def generate_response(model=model_name, result=search_results, user_query=query):
    generator = pipeline("text-generation", model=model_name)

    # Extract relevant information
    matches = result.get("matches", [])
    formatted_results = "\n".join(
        [f"- {match['metadata']['text']} (Score: {match['score']:.2f})" for match in matches]
    ) if matches else "No relevant data found."

    # Updated prompt
    final_prompt = f"""You are a responsible, safety-first AI assistant designed to provide general information on medical knowledge.

User Query: {user_query}

Relevant Information:
{formatted_results}

Now, provide a clear and informative response:

"""

    output = generator(
        final_prompt,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=50256,
        eos_token_id=None,
        return_full_text=False
    )

    return output[0]["generated_text"]

output = generate_response(model_name, search_results, query)
print(output)

Device set to use cpu




What is Polio?

Polio is an acute viral infection due to a poliovirus (serotypes 1, 2 or 3). It is caused by the Poliovirus serotype 1 (PV1), and is transmitted through the fecal-oral route. The virus is spread by person-to-person contact.

The disease is endemic in the world, and is characterized by fever, diarrhea, and a rash. It is also known as the "Polio Fever." In most cases, symptoms are mild and resolve within days. The most common symptoms include:

Fever (temperature greater than 100.4°F)

Diarrhea

Rash

Anorexia

Fatigue

If you are experiencing any of these symptoms, you may be infected with the virus. If you have not been exposed to the virus, your symptoms may be mild. However, if you have been exposed to the virus,
